# A/B Test Challenge



---

#### What is an A/B Test? 

It is a decision making support & research methodology that allow you to measure an impact of a change in a product (e.g.: a digital product). For this challenge you will analyse the data resulting of an A/B test performed on a digital product where a new set of sponsored ads are included.


#### Measure of success

Metrics are needed to measure the success of your product. They are typically split in the following categories: 

- __Enganged based metrics:__ number of users, number of downloads, number of active users, user retention, etc.

- __Revenue and monetization metrics:__ ads and affiliate links, subscription-based, in-app purchases, etc.

- __Technical metrics:__ service level indicators (uptime of the app, downtime of the app, latency).



---

## Metrics understanding

In this part you must analyse the metrics involved in the test. We will focus in the following metrics:

- Activity level + Daily active users (DAU).

- Click-through rate (CTR)

### Activity level

In the following part you must perform every calculation you consider necessary in order to answer the following questions:

- How many activity levels you can find in the dataset (Activity level of zero means no activity).

- What is the amount of users for each activity level.

- How many activity levels do you have per day and how many records per each activity level.

At the end of this section you must provide your conclusions about the _activity level_ of the users.

__Dataset:__ `activity_pretest.csv`

In [1]:
import pandas as pd
from zipfile import ZipFile
import numpy as np
from statsmodels.stats.weightstats import ztest
from scipy import stats
import seaborn as sns
import matplotlib.pylab as plt

In [2]:
# your-code

activity_pretest = pd.read_csv('./data/activity_pretest.csv')
activity_pretest.head()

# Guille: bad4cc056

,userid,dt,activity_level
0,a5b70ae7-f07c-4773-9df4-ce112bc9dc48,2021-10-01,0
1,d2646662-269f-49de-aab1-8776afced9a3,2021-10-01,0
2,c4d1cfa8-283d-49ad-a894-90aedc39c798,2021-10-01,0
3,6889f87f-5356-4904-a35a-6ea5020011db,2021-10-01,0
4,dbee604c-474a-4c9d-b013-508e5a0e3059,2021-10-01,0


In [3]:
activity_pretest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860000 entries, 0 to 1859999
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   userid          object
 1   dt              object
 2   activity_level  int64 
dtypes: int64(1), object(2)
memory usage: 42.6+ MB


In [4]:
# Eliminar activity_level == 0
activity_pretest = activity_pretest[activity_pretest['activity_level'] != 0].reset_index(drop=True)
activity_pretest

,userid,dt,activity_level
0,428070b0-083e-4c0e-8444-47bf91e99fff,2021-10-01,1
1,93370f9c-56ef-437f-99ff-cb7c092d08a7,2021-10-01,1
2,0fb7120a-53cf-4a51-8b52-bf07b8659bd6,2021-10-01,1
3,ce64a9d8-07d9-4dca-908d-5e1e4568003d,2021-10-01,1
4,e08332f0-3a5c-4ed2-b957-87e464e89b97,2021-10-01,1
...,...,...,...
950870,200d65e6-b1ce-4a47-8c2b-946db5c5a3a0,2021-10-31,20
950871,535dafe4-de7c-4b56-acf6-aa94f21653bc,2021-10-31,20
950872,0428ca3c-e666-4ef4-8588-3a2af904a123,2021-10-31,20
950873,a8cd1579-44d4-48b3-b3d6-47ae5197dbc6,2021-10-31,20


In [5]:
# How many activity levels you can find in the dataset (Activity level of zero means no activity).

print(activity_pretest['activity_level'].unique()) 
print(activity_pretest['activity_level'].nunique()) # There are 20 activity levels

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
20


In [13]:
# What is the amount of users for each activity level.

users_per_activity = activity_pretest.groupby('activity_level')['userid'].nunique()
users_per_activity

activity_level
1     33688
2     33761
3     33634
4     33502
5     33820
6     33789
7     33337
8     33365
9     33636
10    33784
11    33730
12    33649
13    33586
14    33703
15    33409
16    33831
17    33333
18    33668
19    33756
20    20215
Name: userid, dtype: int64

In [17]:
# How many activity levels do you have per day and how many records per each activity level.
daily_act_levels = activity_pretest.groupby(['dt', 'activity_level']).count()
daily_act_levels

userid
dt         activity_level        
2021-10-01 1                 1602
           2                 1507
           3                 1587
           4                 1551
           5                 1586
...                           ...
2021-10-31 16                1499
           17                1534
           18                1531
           19                1616
           20                 783

[620 rows x 1 columns]

### Daily active users (DAU)

![ab_test](./img/user_activity_ab_testing.JPG)


The daily active users (DAU) refers to the amount of users that are active per day (activity level of zero means no activity). You must perform the calculation of this metric and provide your insights about it.

__Dataset:__ `activity_pretest.csv`

In [19]:
activity_pretest.head()

,userid,dt,activity_level
0,428070b0-083e-4c0e-8444-47bf91e99fff,2021-10-01,1
1,93370f9c-56ef-437f-99ff-cb7c092d08a7,2021-10-01,1
2,0fb7120a-53cf-4a51-8b52-bf07b8659bd6,2021-10-01,1
3,ce64a9d8-07d9-4dca-908d-5e1e4568003d,2021-10-01,1
4,e08332f0-3a5c-4ed2-b957-87e464e89b97,2021-10-01,1


In [25]:
# your-code


# How many active users per day and how many records per each activity level.
daily_act_users = activity_pretest.groupby(['dt', 'userid']).count()
daily_act_users
# How many activity levels do you have per day and how many records per each activity level.
# daily_act_levels = activity_pretest.groupby(['dt', 'activity_level']).count()
# daily_act_levels
'''Pendiente de revisar'''

'Pendiente de revisar'

### Click-through rate (CTR)

![ab_test](./img/ad_click_through_rate_ab_testing.JPG)

Click-through rate (CTR) refers to the percentage of clicks that the user perform from the total amount ads showed to that user during a certain day. You must perform the analysis of this metric (e.g.: average CTR per day) and provide your insights about it.

__Dataset:__ `ctr_pretest.csv`

In [26]:
ctr_pretest = pd.read_csv('./data/ctr_pretest.csv')
ctr_pretest

,userid,dt,ctr
0,4b328144-df4b-47b1-a804-09834942dce0,2021-10-01,34.28
1,34ace777-5e9d-40b3-a859-4145d0c35c8d,2021-10-01,34.67
2,8028cccf-19c3-4c0e-b5b2-e707e15d2d83,2021-10-01,34.77
3,652b3c9c-5e29-4bf0-9373-924687b1567e,2021-10-01,35.42
4,45b57434-4666-4b57-9798-35489dc1092a,2021-10-01,35.04
...,...,...,...
950870,a09a3687-b71a-4a67-b1ef-9b05c9770c4c,2021-10-31,32.33
950871,c843a595-b94c-42e1-b2fe-ec096070681e,2021-10-31,30.09
950872,edcdf0c1-3d8f-47e8-b7dd-05505749eb69,2021-10-31,35.71
950873,76b7a9ae-98fa-4c77-869d-594a4ef7282d,2021-10-31,34.76


In [27]:
# your-code




---

## Pretest metrics 

In this section you will perform the analysis of the metrics using the dataset that includes the result for the test and control groups, but only for the pretest data (i.e.: prior to November 1st, 2021). You must provide insights about the metrics (__Activity level__, __DAU__ and __CTR__) and also perform an hyphotesis test in order to determine whether there is any statistical significant difference between the groups prior to the start of the experiment. You must try different approaches (i.e.: __z-test__ and __t-test__) and compare the results.


__Datasets:__ `activity_all.csv`, `ctr_all.csv`

In [10]:
# your-code

activity_all = pd.read_csv('./data/activity_all.csv')
activity_all


,userid,dt,groupid,activity_level
0,a5b70ae7-f07c-4773-9df4-ce112bc9dc48,2021-10-01,0,0
1,d2646662-269f-49de-aab1-8776afced9a3,2021-10-01,0,0
2,c4d1cfa8-283d-49ad-a894-90aedc39c798,2021-10-01,1,0
3,6889f87f-5356-4904-a35a-6ea5020011db,2021-10-01,0,0
4,dbee604c-474a-4c9d-b013-508e5a0e3059,2021-10-01,1,0
...,...,...,...,...
3659995,f0126b50-ad74-4480-9250-41b50a408932,2021-11-30,0,20
3659996,6ffe1efe-2e5d-427f-95ff-cc862c46c798,2021-11-30,1,20
3659997,f2073207-25dd-4127-a893-b70106d5ead7,2021-11-30,0,20
3659998,0416f2be-3ab8-481b-873c-3678b4705ecf,2021-11-30,1,20


In [11]:
# your-code
ctr_all = pd.read_csv('./data/ctr_all.csv')
ctr_all['dt'] = pd.to_datetime(ctr_all['dt']), for
mat = 

SyntaxError: invalid syntax (2751942850.py, line 3)

---

## Experiment metrics 

In this section you must perform the same analysis as in the previous section, but using the data generated during the experiment (i.e.: after November 1st, 2021). You must provide insights about the metrics (__Activity level__, __DAU__ and __CTR__) and also perform an hyphotesis test in order to determine whether there is any statistical significant difference between the groups during the experiment. You must try different approaches (i.e.: __z-test__ and __t-test__) and compare the results.


__Datasets:__ `activity_all.csv`, `ctr_all.csv`

In [ ]:
# your-code




---

## Conclusions

Please provide your conclusions after the analyses and your recommendation whether we may or may not implement the changes in the digital product.

In [ ]:
# your-conclusions




---